In [1]:
from math import sqrt

In [2]:
import numpy as np

In [3]:
from geometry import triangulate

In [4]:
from helpers import make_mesh

ModuleNotFoundError: No module named 'stl'

In [53]:
EIGHTH = [
    [(0, 0, 2), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 1, 1), (1, 1, 1)],
    [(0, 2, 1), (0, 2, 0), (1, 1, 1)],
    [(2, 2, 0), (0, 2, 0), (1, 1, 1)],
]   

In [54]:
def transform(shape, fn):
    return [[fn(x, y, z) for x, y, z in tri] for tri in shape]

In [139]:
Z_SCALE = sqrt(2)

In [141]:
QUARTER = EIGHTH + transform(EIGHTH, lambda x, y, z: (y, x, z))

In [147]:
WALL = 0.4
FLOOR = 0.6

In [148]:
HEIGHT = 2 * Z_SCALE + FLOOR

In [149]:
SIZE = 2 + WALL

In [150]:
INNER_QUARTER = transform(QUARTER, lambda x, y, z: (x, y, HEIGHT - Z_SCALE * z))

In [151]:
INNER_QUARTER

[[(0, 0, 0.6000000000000001),
  (0, 1, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(0, 2, 2.0142135623730955),
  (0, 1, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(0, 2, 2.0142135623730955),
  (0, 2, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(2, 2, 3.4284271247461904),
  (0, 2, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(0, 0, 0.6000000000000001),
  (1, 0, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(2, 0, 2.0142135623730955),
  (1, 0, 2.0142135623730955),
  (1, 1, 2.0142135623730955)],
 [(2, 0, 2.0142135623730955),
  (2, 0, 3.4284271247461904),
  (1, 1, 2.0142135623730955)],
 [(2, 2, 3.4284271247461904),
  (2, 0, 3.4284271247461904),
  (1, 1, 2.0142135623730955)]]

In [152]:
QUARTER_FRONT_TRI_RAW = [
    [(0, 1, 1), (0, 0, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 2), (0, 1, 2)],
    [(0, 1, 1), (0, 2, 1), (0, 2, 2)],
]

QUARTER_FRONT_TRI = transform(QUARTER_FRONT_TRI_RAW, lambda x, y, z: (x, y, HEIGHT - Z_SCALE * z))

FRONT_BOTTOM_RECT = [
    (0, 0, 0), (0, 0, FLOOR), (0, 2, FLOOR), (0, 2, 0)
]

QUARTER_FRONT = QUARTER_FRONT_TRI + list(triangulate(FRONT_BOTTOM_RECT))

In [153]:
MOULD_QUARTER = INNER_QUARTER + QUARTER_FRONT

In [154]:
RIM_QUARTER = list(triangulate(
    [(0, 2, HEIGHT), (0, SIZE, HEIGHT), (SIZE, SIZE, HEIGHT), (2, 2, HEIGHT)]
))

TOP_RIM = (
    RIM_QUARTER
    + transform(RIM_QUARTER, lambda x, y, z: (y, x, z))
    + transform(RIM_QUARTER, lambda x, y, z: (x, -y, z))
    + transform(RIM_QUARTER, lambda x, y, z: (y, -x, z))
)

In [155]:
FLOOR_SQUARE = list(triangulate(
    [(0, 0, 0), (0, 2, 0), (2, 2, 0), (2, 0, 0)]
))

FLOOR_RIM = transform(TOP_RIM, lambda x, y, z: (x, y, 0))

FLOOR_ALL = FLOOR_RIM + FLOOR_SQUARE

In [156]:
heights = [0, FLOOR, FLOOR + Z_SCALE, FLOOR + 2 * Z_SCALE]

def wall_pieces(base1, base2):
    x1, y1 = base1
    x2, y2 = base2

    for z1, z2 in zip(heights[:-1], heights[1:]):
        yield from triangulate([(x1, y1, z1), (x1, y1, z2), (x2, y2, z2), (x2, y2, z1)])

In [157]:
LEFT_OUTER_PIECES = [
    [(0, 2), (0, SIZE)],
    [(0, SIZE), (SIZE, SIZE)],
    [(SIZE, SIZE), (SIZE, 0)],
]

OUTER_PIECES = LEFT_OUTER_PIECES + [[(x, -y) for x, y in piece] for piece in LEFT_OUTER_PIECES]

WALL_OUTER = list(tri for piece in OUTER_PIECES for tri in wall_pieces(*piece))

In [158]:
WRAPPER = FLOOR_ALL + WALL_OUTER + TOP_RIM

In [159]:
OUTER_MOULD = MOULD_QUARTER + WRAPPER

In [160]:
DIVISION = (
    transform(QUARTER_FRONT, lambda x, y, z: (y, x, z))
    + list(wall_pieces((2, 0), (2, -2)))
    + list(wall_pieces((2, -2), (0, -2)))
)

In [161]:
PIECE_4 = OUTER_MOULD + DIVISION

In [162]:
MM_SCALE = 15

In [163]:
def write_mesh(shape, scale, filename):
    data = np.array(shape)
    make_mesh(data * scale).save(filename)

In [164]:
write_mesh(PIECE_4, MM_SCALE, 'piece_4.stl')

In [165]:
FILLER_SQUARE = transform(FLOOR_SQUARE, lambda x, y, z: (x, -y, z))
FILLER_MOULD_QUARTER = transform(MOULD_QUARTER, lambda x, y, z: (x, -y, z))

FILLER = FILLER_SQUARE + FILLER_MOULD_QUARTER

In [166]:
PIECE_5 = FILLER + DIVISION

In [167]:
write_mesh(PIECE_5, MM_SCALE, 'piece_5.stl')

In [168]:
PIECE_1 = OUTER_MOULD + FILLER

In [169]:
write_mesh(PIECE_1, MM_SCALE, 'piece_1.stl')